In [1]:
import tensorflow as tf

# Create tf dataset from a list

In [2]:
daily_sales_numbers = [21,22,-108,31,-1,32,34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

Metal device set to: Apple M1


2023-02-17 09:33:59.056398: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-17 09:33:59.056859: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

# Iterate through tf dataset

In [3]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


# Iterate through elements as numpy elements

In [5]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


# Filter sales numbers that are < 0

In [6]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


2023-02-17 09:38:50.750178: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


# Convert sales numbers from USA dollars to Indian Rupees Assuming 1->72 conversation rate

In [7]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


# Shuffle

In [8]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1584
1512
2304
2448
2232
2232


# Batching

In [9]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1512 2232]
[1584 2304]
[2232 2448]


# Perform all of the above operations in one shot

In [10]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1512 1584]
[2304 2448]
[2232 2232]


# Images

In [11]:
images_ds = tf.data.Dataset.list_files("images/*/*", shuffle=False)

In [12]:
images_count = len(images_ds)
images_count

130

In [13]:
type(images_ds)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [15]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'images/dog/Best Hypoallergenic Dogs [Updated....jpg'
b'images/dog/Germany_ Dogs must be walked twice a....jpg'
b'images/dog/10 Teacup Dog Breeds for Tiny Canine Lovers.jpg'


In [16]:
class_names = ["cat","dog"]

In [18]:
train_size = int(images_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [19]:
len(train_ds)

104

In [20]:
len(test_ds)

26

In [47]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [48]:
get_label("images/dog/20 Reasons Why Cats Make the Best Pets....jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'dog'>

In [49]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128,128])
    return img, label

In [50]:
img, label = process_image("images/cat/20 Reasons Why Cats Make the Best Pets....jpg")
img.numpy()[:2]

array([[[155.      , 186.      , 215.      ],
        [156.      , 187.      , 216.      ],
        [158.      , 189.      , 218.      ],
        [160.0039  , 189.0039  , 219.0039  ],
        [161.0039  , 190.0039  , 220.0039  ],
        [162.      , 191.      , 221.      ],
        [166.      , 193.      , 222.      ],
        [167.      , 194.      , 223.      ],
        [168.      , 195.      , 224.      ],
        [169.      , 196.      , 225.      ],
        [170.      , 197.      , 224.      ],
        [170.0039  , 197.0039  , 224.0039  ],
        [172.      , 199.      , 226.      ],
        [173.0039  , 199.0039  , 224.0039  ],
        [174.0039  , 200.0039  , 225.0039  ],
        [175.0039  , 201.0039  , 226.0039  ],
        [176.      , 202.      , 227.      ],
        [177.0039  , 203.0039  , 228.0039  ],
        [177.0039  , 203.0039  , 228.0039  ],
        [179.0039  , 203.0039  , 227.0039  ],
        [180.0039  , 204.0039  , 228.0039  ],
        [180.0039  , 204.0039  , 2

In [51]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [52]:
for image, label in train_ds.take(1):
    print("****", image)
    print("****", label)

**** tf.Tensor(
[[[197.22656 189.22656 187.22656]
  [187.22656 177.22656 175.22656]
  [195.9336  186.9336  181.9336 ]
  ...
  [181.58594 166.58594 162.58594]
  [182.16016 171.16016 167.16016]
  [177.72656 166.72656 164.72656]]

 [[192.17969 184.17969 181.17969]
  [192.14062 182.14062 180.14062]
  [195.83984 186.51953 182.48047]
  ...
  [181.23828 166.23828 162.5586 ]
  [179.66016 168.66016 164.66016]
  [175.85938 163.89844 162.21875]]

 [[191.9336  184.80078 179.19922]
  [194.4336  186.16797 183.30078]
  [200.60156 190.60156 189.03516]
  ...
  [177.23438 162.23438 159.23438]
  [185.10156 174.10156 170.10156]
  [170.9336  156.9336  154.19922]]

 ...

 [[198.46875 187.60156 185.60156]
  [192.69922 182.5664  181.4336 ]
  [203.0664  195.0664  193.     ]
  ...
  [158.39844 145.13281 145.0664 ]
  [160.60156 148.33594 148.33594]
  [162.13281 150.73438 149.     ]]

 [[196.67969 186.67969 184.67969]
  [190.1211  179.1211  177.1211 ]
  [201.1211  193.1211  190.1211 ]
  ...
  [151.40234 135.40234

In [53]:
def scale(image, label):
    return image/255, label

In [54]:
train_ds = train_ds.map(scale)

In [55]:
for image, label in train_ds.take(5):
    print("****Image:", image.numpy()[0][0])
    print("****Label:", label.numpy())

****Image: [0.00292969 0.3957376  0.26240426]
****Label: b'cat'
****Image: [0.31587416 0.33235702 0.10178103]
****Label: b'dog'
****Image: [0.9098039  0.84313726 0.77254903]
****Label: b'dog'
****Image: [0.1464614  0.13469669 0.00528493]
****Label: b'dog'
****Image: [0.8784314 0.8156863 0.7647059]
****Label: b'dog'
